## 瀏覽器&套件設定

In [1]:
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

/Users/ponywen/Documents/projects/Facebook_scaper/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# 擋掉通知黑屏，以防止瀏覽器第一次登入時跳出通知視窗
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.notifications": 2
})

In [3]:
options.add_argument('--headless') # colab上須加上headless，表不使用GUI背景執行
options.add_argument('--no-sandbox')
options.headless = True

## 自動登入

In [4]:
# ------ 登入的帳號與密碼 ------
# username = str(input("請輸入帳號："))
username = "xiao901202@gmail.com"
# password = str(input("請輸入密碼："))
password="qwerasdf520"
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

url = 'https://www.facebook.com'  
driver.get(url)

WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="email"]')))

elem = driver.find_element(By.ID,"email")
elem.send_keys(username)

elem = driver.find_element(By.ID,"pass")
elem.send_keys(password)


# 提交登入
elem.send_keys(Keys.RETURN)
time.sleep(5)

# 檢查有沒有被擋下來
if len(driver.find_elements(By.XPATH,"//*[contains(text(), '你的帳號暫時被鎖住')]")) > 0:
    driver.find_elements(By.XPATH,"//*[contains(text(), '是')]")[1].click()
elif len(driver.find_elements(By.XPATH, "//*[contains(text(), '用戶名稱或密碼無效')]")) > 0:
    print("登入失敗，請檢查帳號密碼是否正確")
elif len(driver.find_elements(By.XPATH, "//*[contains(text(), 'Invalid username or password')]")) > 0:
    print("登入失敗，請檢查帳號密碼是否正確")
else:
    feed = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="main"]'))) # 等待動態頁面(鎖定貼文)加載完成
    print("登入成功")

登入成功


## 純貼文爬取

In [5]:
import base64


def image_to_data_url(image_path: str):
    # 開啟圖片文件
    with open(image_path, 'rb') as image_file:
        # 讀取圖片數據
        image_data = image_file.read()

    # 獲取 MIME 類型（這裡以 PNG 為例，根據你的圖片類型進行調整）
    mime_type = 'image/png' if image_path.endswith('.png') else 'image/jpeg'

    # 編碼圖片數據
    base64_encoded_data = base64.b64encode(image_data)

    # 轉換為字符串並格式化為 Data URL
    base64_data_url = f'data:{mime_type};base64,{base64_encoded_data.decode()}'

    return base64_data_url

In [7]:
from openai import OpenAI

key = ''
with open("key.txt", "r") as f:
    key = f.read()
    f.close()

client = OpenAI(api_key=key)


def get_post_category(image_url):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": "這是我設計的臉書可能有之貼文類型：\n政治包括對政策、政治事件、選舉和政治人物的討論或意見\n心情/情感分享 分享個人的情緒狀態、心情好壞、生活中的盧受和經歷·\n廣告/推廣 包含推銷產品、服務咸活動的信息·可能來自個人或企業帳號·\n生活日常 分享日常生活的點滴·如日常活動、家庭生活、個人愛好 等\n新聞/時事 談論時事新聞、社會事件、重大發生等\n教育/學習 分享教育相開內容、學習資源、課程信息等·\n勵志/名言 包含勵志語錄、名人名宮、個人創造的正面能量語句\n問題諮詢 尋求幫助、建議、意見交流的貼文、涉及生活、工作、情感 等方面的問題\n活動邀請/公告 包括聚會邀請·活動宣傳、特別公告等\n旅遊分享 分享旅行經驗、旅遊計劃或心得·\n美食分享 談論美食、分享食譜、餐觀推薦等\n藝術/創作 分享個人的創作、藝衛作品、文學作品等\n健康/運動 談論健康話題、運動心得、健身技巧等·\n科技/數碼 分享科技新聞、數碼產品評瀾、使用心得等·\n商業/經濟 討論商業動態·經濟趨勢、市塌分析等·\n宗教/信仰分享宗教信仰相關的釀悟、活動·節日信息等·\n娛樂 \n\n接下來我會貼給你一個臉書貼文，請直接告訴我覺得是哪一個不要有其他任何文字"
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {"url": image_url}
                    }
                ]
            }
        ],
        temperature=0,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

In [9]:
import time

posts_number = int(input("請輸入要爬取的貼文數量:"))
posts = [] 

driver.set_window_size(1000,4000)
# 初始設定
last_height = driver.execute_script("return document.body.scrollHeight") # 取得頁面高度初始值
last_count = 0 # 計算目前貼文數量

feed = driver.find_element(By.CSS_SELECTOR, 'div[role="main"]') # 找到動態頁面(鎖定貼文)
while True:
    # 如果已經找到足夠的貼文，則停止
    if len(posts) >= posts_number:
        posts = posts[:posts_number]
        break
    
    # 貼文文字的 class selector
    post_text_class_selector = 'div[data-ad-preview="message"]'
    
    # 捕捉所有貼文元素
    
    elements = feed.find_elements(By.CSS_SELECTOR, 'div[class = x1lliihq]')
    current_count = len(elements)    
    if current_count == 0:
        elements = feed.find_elements(By.CSS_SELECTOR, '[data-pagelet]')
        current_count = len(elements)
    print(last_count,current_count)
    

    if current_count != last_count:
        try:
            print("!!! 檢查當前是否有 「查看更多」 Button !!!")
            buttons = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(text(), '查看更多') and @role='button']")))
            for button in buttons:

                # 滾動到按鈕位置
                driver.execute_script("arguments[0].scrollIntoView(true);", button)
                WebDriverWait(driver, 5).until(EC.visibility_of(button))
                # 使用 JavaScript 強制點擊按鈕
                driver.execute_script("arguments[0].click();", button)
                print('Button clicked!')
        except Exception as e:
            print(f"當前無「查看更多」 Button")
            pass

        print('!!! 檢查完畢 !!!')
        while len(posts) < posts_number and len(elements) > last_count
            current_element = elements[last_count]
            last_count += 1
            try:
                print('--------------貼文分隔線--------------')
                element_post = current_element.find_element(By.CSS_SELECTOR, post_text_class_selector)
                post_text = element_post.text
                current_element.screenshot('element_screenshot.png')
                print("元素截圖完成")
                screenshot_base64 = image_to_data_url('element_screenshot.png')
                category = get_post_category(screenshot_base64)
                posts.append({
                    'text': post_text, 'image': screenshot_base64, 'category': category
                })
                print(len(posts),post_text,category )
            except Exception as e:
                print(e)
                print('此篇非普通貼文，跳過處理')
        print('---------------------------------')

    # 計算新的滾動高度並比較是否到達底部
    new_height = driver.execute_script("return document.body.scrollHeight")
    last_height = new_height

    # 滾動到頁面底部
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 等待新內容加載(秒)
    time.sleep(4)

print('共爬取', len(posts), '篇貼文')
print(posts)
# driver.refresh()

0 6
!!! 檢查當前是否有 「查看更多」 Button !!!
Button clicked!
Button clicked!
!!! 檢查完畢 !!!
--------------貼文分隔線--------------
Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-ad-preview="message"]"}
  (Session info: chrome-headless-shell=126.0.6478.62); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104a67a38 chromedriver + 4389432
1   chromedriver                        0x0000000104a60344 chromedriver + 4358980
2   chromedriver                        0x000000010467cafc chromedriver + 281340
3   chromedriver                        0x00000001046bf2c8 chromedriver + 553672
4   chromedriver                        0x00000001046b5718 chromedriver + 513816
5   chromedriver                        0x00000001046f7cec chromedriver + 785644
6   chromedriver                        0x00000001046b3ed0 chrome

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [10]:
print(len(posts))

10


In [11]:
import csv
import os

def save_posts_to_csv(posts, filename):
    file_exists = os.path.exists(filename)

    # 打开文件，准备写入
    with open(filename, mode='a', newline='', encoding='utf-8-sig') as file:
        
        # 如果文件不存在，添加标题行
        if not file_exists:
            file.write("\"Text\",\"Image\",\"Category\"\n")
            # file.write("\"Text\",\"Category\"\n")
        # 遍历帖子列表，写入每条帖子数据
        for post in posts:
            processed_text = post['text'].replace("\n"," ")
            file.write(f"\"{processed_text}\",\"{post['image']}\",\"{post['category']}\"\n")
            
            # file.write(f"\"{post['text']}\",\"{post['category']}\"\n")

# 调用函数，保存数据到 CSV
save_posts_to_csv(posts, 'res.csv')


In [20]:
from PIL import Image

width = driver.execute_script("return document.documentElement.scrollWidth")
height = driver.execute_script("return document.documentElement.scrollHeight")
driver.set_window_size(width,height)

feed = driver.find_element(By.CSS_SELECTOR, 'div[role="main"]') # 找到動態頁面(鎖定貼文)
print('网页模块尺寸:height={},width={}'.format(feed.size['height'],feed.size['width']))

feed.screenshot('feed.png')
im=Image.open('feed.png')
print("截图尺寸:height={},width={}".format(im.size[1],im.size[0]))

网页模块尺寸:height=2084,width=603
截图尺寸:height=1558,width=670


In [44]:
import time

current_timestamp_ms = int(time.time() * 1000)
print(f"Current Timestamp in milliseconds: {current_timestamp_ms}")

Current Timestamp in milliseconds: 1718200121257
